# KDD 2009 - Tratamento de dados simplista
Como parte do exercício de resolução do KDD Cup de 2009 criamos este notebook para realizar um teste do modelo preditivo sem fazer grandes alterações nos dados originais. A análise completa pode ser vista [aqui](./KDD2009.ipynb).

O objetivo é ter um parâmetro de comparação com o modelo entregue, que utiliza o **vtreat** para gerar um plano de tratamento das variáveis do KDD Cup 2009, para cada uma dos resultados desejados (*appetency*, *churn* e *up-selling*).

Os únicos tratamentos serão:
- Remoção das variáveis nulas (sem nenhum valor válido)
- Conversão das variáveis categóricas para valores numéricos

Os valores NA do conjunto de dados serão mantidos como estão.

In [1]:
# Importação de bibliotecas
import math
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import random
from sklearn.metrics import auc
import xgboost as xgb

# Lê arquivo de treinamento

In [2]:
# Diretório de dados
data_folder = os.path.join("..", "data")
# Arquivo de dados
training_data_file = os.path.join(data_folder, "orange_small_train.data")

# Dataframae com os dados de treinamento
kdd2009 = pd.read_csv(training_data_file, delimiter='\t', na_values=[''])


## Cria dataframe de contagem de NaN e remove colunas que só contém NaN

In [3]:
# Número total de pontos de dados
numeroDados = len(kdd2009)

# Cria dataframe com contagem de NaNs
nans = pd.DataFrame(kdd2009.isna().sum(), columns=['NAN_count'])
# Recupera lista de variaveis que só possuem NaN
variaveisNulas = nans[nans['NAN_count'] == numeroDados].index.values

# Remove as colunas que só contém NaNs
kdd2009.drop(variaveisNulas, axis=1, inplace=True)

# Lê arquivos de labels
Lê arquivos com os rótulos de *Appetency, Churn* e *Upselling*. Inclui labels no dataframe do kdd2009

In [4]:
# Arquivos com os rótulos das métricas
# Colunas de resultados
resultados = ['appetency', 'churn', 'upselling']
padraoArquivosRotulos = "orange_small_train_{}.labels"

# Colunas de variáveis de entrada
variaveis = kdd2009.keys()

# Le os arquivos de rótulos, converte 1 -> True, -1 -> False e inclui no dataframe do kdd
for rotulo in resultados:
    # Gera nome do arquivo
    nomeArquivo = os.path.join(data_folder, padraoArquivosRotulos.format(rotulo))
    # Lê arquivo
    dfRotulo = pd.read_csv(nomeArquivo, header=None, names=['label'])
    # Converte 1 -> True, -1 -> False - Assume que != 1 -> False
    dfRotulo['label'] = dfRotulo['label'] == 1
    # Inclusão dos rótulos no dataframe do kdd como colunas adicionais
    kdd2009[rotulo] = dfRotulo['label']

# Verificando os rótulos

In [5]:
# Primeiros valores dos resultados no dataframe de treinamento
kdd2009[resultados].head()

,appetency,churn,upselling
0,False,False,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


## E as variáveis (já sem colunas de NaNs)

In [6]:
# Primeiros valores das variaveis no datarame de treinamento
kdd2009[variaveis].head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


In [7]:
list(variaveis)

['Var1',
 'Var2',
 'Var3',
 'Var4',
 'Var5',
 'Var6',
 'Var7',
 'Var9',
 'Var10',
 'Var11',
 'Var12',
 'Var13',
 'Var14',
 'Var16',
 'Var17',
 'Var18',
 'Var19',
 'Var21',
 'Var22',
 'Var23',
 'Var24',
 'Var25',
 'Var26',
 'Var27',
 'Var28',
 'Var29',
 'Var30',
 'Var33',
 'Var34',
 'Var35',
 'Var36',
 'Var37',
 'Var38',
 'Var40',
 'Var41',
 'Var43',
 'Var44',
 'Var45',
 'Var46',
 'Var47',
 'Var49',
 'Var50',
 'Var51',
 'Var53',
 'Var54',
 'Var56',
 'Var57',
 'Var58',
 'Var59',
 'Var60',
 'Var61',
 'Var62',
 'Var63',
 'Var64',
 'Var65',
 'Var66',
 'Var67',
 'Var68',
 'Var69',
 'Var70',
 'Var71',
 'Var72',
 'Var73',
 'Var74',
 'Var75',
 'Var76',
 'Var77',
 'Var78',
 'Var80',
 'Var81',
 'Var82',
 'Var83',
 'Var84',
 'Var85',
 'Var86',
 'Var87',
 'Var88',
 'Var89',
 'Var90',
 'Var91',
 'Var92',
 'Var93',
 'Var94',
 'Var95',
 'Var96',
 'Var97',
 'Var98',
 'Var99',
 'Var100',
 'Var101',
 'Var102',
 'Var103',
 'Var104',
 'Var105',
 'Var106',
 'Var107',
 'Var108',
 'Var109',
 'Var110',
 'Var11

# Classificação de colunas
Classifica as colunas em inteiros, reais, objetos e booleanos

In [8]:
# Verifica os tipos de dados existentes no dataframe
print('Lista de tipos de dados no kdd2009: {}'.format(list(set(kdd2009[variaveis].dtypes))))

Lista de tipos de dados no kdd2009: [dtype('float64'), dtype('O'), dtype('int64')]


In [9]:
data_types = pd.DataFrame(kdd2009[variaveis].dtypes, columns=['dtype'])

variaveisInteiras = list(data_types[data_types.values == np.dtype('int64')].index.values)
variaveisReais = list(data_types[data_types.values == np.dtype('float64')].index.values)
variaveisCategoricas = list(data_types[data_types.values == np.dtype('object')].index.values)

len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) == len(kdd2009.keys())

print('Há {} variáveis inteiras, {} variáveis reais e {} objetos'.format(
                                                                    len(variaveisInteiras),
                                                                    len(variaveisReais),
                                                                    len(variaveisCategoricas)))

# Verifica se o total de tipos de variáveis está correto
if len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) != len(kdd2009.keys()):
    print('Aviso: erro na contagem de tipos de variáveis!!!')

Há 1 variáveis inteiras, 173 variáveis reais e 38 objetos


In [10]:
kdd2009[variaveisCategoricas].head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,CEat0G8rTN,RO12,NaN,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,jg69tYsGvO,RO12,NaN,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


In [11]:
kdd2009.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,702.000000,1.487000e+03,1240.000000,...,7.020000e+02,44991.000000,1.579000e+03,1.241000e+03,1241.000000,702.000000,702.00000,1241.000000,21022.000000,333.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,...,3.776755e+06,0.611456,1.416638e+06,7.777380e+04,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,154.777855,9.280896e+05,2.869558,...,3.785696e+06,2.495681,2.279786e+06,2.016188e+05,46.973777,8.781967,60.22303,113.980072,86.707692,29085.146490
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,-6.420000,6.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,4.000000,0.000000e+00,8.000000,...,1.917352e+05,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,19.380000,204.000000,2732.670000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,20.000000,0.000000e+00,8.000000,...,2.431310e+06,0.000000,1.167780e+05,0.000000e+00,0.000000,0.000000,4.00000,197.640000,270.000000,12668.940000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,46.000000,2.628630e+05,8.000000,...,6.471827e+06,0.000000,1.844952e+06,4.881000e+04,8.000000,6.000000,14.00000,252.960000,330.000000,29396.340000
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,...,1.428483e+07,49.000000,1.199478e+07,3.048400e+06,1200.000000,102.000000,910.00000,628.620000,642.000000,230427.000000


In [12]:
kdd2009.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,False,False,False
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,False,True,False
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,False,False,False
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,False,False,False


In [13]:
pd.Categorical(kdd2009[variaveisCategoricas[-1]])

[NaN, NaN, am7c, NaN, mj86, ..., NaN, am7c, NaN, NaN, NaN]
Length: 50000
Categories (4, object): [am7c, mj86, oJmt, sk2h]

## Converte variáveis categóricas para inteiros (para usar no XGBoost)

In [14]:
for variavel in variaveisCategoricas:
    kdd2009[variavel] = pd.Categorical(kdd2009[variavel])
    kdd2009[variavel] = kdd2009[variavel].cat.codes

In [15]:
# Verifica os tipos de dados existentes no dataframe
print('Lista de tipos de dados no kdd2009: {}'.format(list(set(kdd2009[variaveis].dtypes))))

Lista de tipos de dados no kdd2009: [dtype('int16'), dtype('float64'), dtype('int8'), dtype('int64')]


In [16]:
kdd2009.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,3,-1,-1,22,2,8,-1,False,False,False
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,0,-1,-1,14,2,8,-1,False,True,False
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,3,-1,1,10,0,25,0,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,0,-1,-1,7,2,8,-1,False,False,False
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,0,-1,1,7,2,8,1,False,False,False


In [17]:
kdd2009.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,702.000000,1.487000e+03,1240.000000,...,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,...,2003.12184,3.984080,2123.702060,0.302480,-0.983600,-0.220100,10.115720,2.103060,10.233400,-0.368800
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,154.777855,9.280896e+05,2.869558,...,1248.04682,1.119166,1183.541455,1.070238,0.127009,0.933506,5.798438,0.867994,6.488254,0.799074
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,...,0.00000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,4.000000,0.000000e+00,8.000000,...,826.00000,4.000000,1066.000000,0.000000,-1.000000,-1.000000,6.000000,2.000000,8.000000,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,20.000000,0.000000e+00,8.000000,...,2014.00000,4.000000,2230.000000,0.000000,-1.000000,-1.000000,10.000000,2.000000,8.000000,-1.000000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,46.000000,2.628630e+05,8.000000,...,3047.00000,4.000000,2984.250000,0.000000,-1.000000,1.000000,14.000000,2.000000,8.000000,0.000000
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,...,4290.00000,6.000000,4290.000000,3.000000,0.000000,2.000000,22.000000,6.000000,29.000000,3.000000


# Divisão do conjunto de dados em treinamento e validação
Os dados originais serão divididos em dois conjunto:

- Treinamento, com 90% dos dados (45.000 dados)
- Validação, com 10% dos dados (5.000 dados)

A divisão será feita da mesma forma (com o mesmo seed) que os dados do modelo com tratamento por **vtreat**.

In [18]:
# Fixa semente para reproduzir resultados
random.seed(42)

# Define treino como 90% dos dados de treino e
# validação como os 10% restantes
tamanhoTreino = math.floor(0.90 * numeroDados)

# Gera uma amostra aleatória com 90% dos índices
indicesTreino = random.sample(range(numeroDados), tamanhoTreino)
indicesValidacao = list(set(range(numeroDados)) - set(indicesTreino))

# Divide os dados nos dois subgrupos
# -indicesTreino são os 10% dos índices restantes
dadosTreino = kdd2009.iloc[indicesTreino].reset_index(drop=True)
dadosValidacao = kdd2009.iloc[indicesValidacao].reset_index(drop=True)

# Dimensões dos dados de treino e validação
print("Dados de treinamento: {}, Dados de validação: {}".format(len(dadosTreino), len(dadosValidacao)))

Dados de treinamento: 45000, Dados de validação: 5000


## Verificando os dataframes

In [19]:
dadosTreino.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,392.0,0.0,NaN,NaN,NaN,...,-1,-1,-1,8,2,8,-1,False,False,False
1,NaN,NaN,NaN,NaN,NaN,1911.0,0.0,NaN,NaN,NaN,...,0,-1,-1,14,2,8,-1,False,False,False
2,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,0,-1,1,12,1,2,0,True,False,False
3,NaN,NaN,NaN,NaN,NaN,2331.0,7.0,NaN,NaN,NaN,...,0,-1,0,17,4,8,0,False,False,False
4,NaN,NaN,NaN,NaN,NaN,119.0,0.0,NaN,NaN,NaN,...,-1,-1,-1,1,0,8,-1,False,False,False


In [20]:
dadosValidacao.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1204.0,0.0,NaN,NaN,NaN,...,0,-1,-1,16,2,8,-1,False,False,False
1,NaN,NaN,NaN,NaN,NaN,805.0,7.0,NaN,NaN,NaN,...,0,-1,0,9,2,8,0,False,False,False
2,NaN,NaN,NaN,NaN,NaN,539.0,7.0,NaN,NaN,NaN,...,0,-1,-1,18,2,8,-1,False,True,False
3,NaN,NaN,NaN,NaN,NaN,1155.0,14.0,NaN,NaN,NaN,...,3,-1,1,4,2,2,0,False,False,False
4,NaN,NaN,NaN,NaN,NaN,693.0,7.0,NaN,NaN,NaN,...,0,-1,-1,7,2,8,-1,False,False,False


## Verificando a distribuição dos rótulos entre treino e validação

In [21]:
dadosTreino[resultados].sum()

appetency     805
churn        3341
upselling    3327
dtype: int64

In [22]:
dadosValidacao[resultados].sum()

appetency     85
churn        331
upselling    355
dtype: int64

# XGBoost

Validação do modelo para churn

In [23]:
resultado = 'churn'
rodadas = 100
xgbDados = xgb.DMatrix(dadosTreino[variaveis], dadosTreino[[resultado]])
xgbTeste = xgb.DMatrix(dadosValidacao[variaveis])

# Exemplo de conversão de dataframe para DMatrix
```
train = pd.read_csv("train.csv")
target = train['target']
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
test = test.drop(['ID'],axis=1)

xgtrain = xgb.DMatrix(train.values, target.values)
xgtest = xgb.DMatrix(test.values)
```

# Exemplo 2
[Docs/XGBoost/Python Package - Python Package Introduction](https://xgboost.readthedocs.io/en/latest/python/python_intro.html)
```
data = pandas.DataFrame(np.arange(12).reshape((4,3)), columns=['a', 'b', 'c'])
label = pandas.DataFrame(np.random.randint(2, size=4))
dtrain = xgb.DMatrix(data, label=label)
```

In [24]:
# Treina o modelo
rodadas=50
# Parâmetros do XGBoost
param = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eta': 0.05,
         'gamma': 0,
         'max_depth':6,
         'min_child_weight': 1,
         'subsample': 1,
         'colsample_bytree': 1}
modelo = xgb.train({#'objective': 'binary:logistic',
                    'max_depth':6,
                    'min_child_weight': 1,
                    'subsample': 1,
                    'colsample_bytree': 1,
                    'num_boost_round':100}, xgbDados, rodadas)
#bst1 = dxgb.train(client, params, X, y)
#bst2 = dxgb.train(client, params, X, y, num_boost_round=100)

[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[00:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[00:42:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[00:42:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[00:42:42] /w

# Computa previsões para o conjunto de validação/teste


In [25]:
resultadoValidacao = modelo.predict(xgbTeste)

validacaoT = dadosValidacao[resultado].apply(int).reset_index(drop=True)
print(validacaoT.head())

# Cria dataframe com rótulo e previsão
df_rV = pd.DataFrame(resultadoValidacao, columns=["resultado"])
df_rV[resultado] = validacaoT
df_rV.head()

0    0
1    0
2    1
3    0
4    0
Name: churn, dtype: int64


,resultado,churn
0,0.069798,0
1,0.020906,0
2,-0.019908,1
3,0.003978,0
4,0.203708,0


In [26]:
from sklearn.metrics import auc

def gradeBoost(treinamento, listaVariaveis, teste, rotulo, intervaloGrade):
    """ Gera lista de resultados da grade de treinamento.
    Parâmetros:
    treinamento: Dataframe de dados de treinamento, com variaveis e rotulos
    listaVariaveis: Lista de variáveis para treinamento/teste
    teste: Dataframe de dados de teste/validação
    rotulo: Rotulo do teste
    intervaloGrade: Range de intervalo da grade de treinamento
    """
    # TODO: Tratamento de parâmetros/datarames de entrada
    
    # Converte dataframes para DMatrices
    xgbDados = xgb.DMatrix(treinamento[listaVariaveis], treinamento[[rotulo]])
    xgbTeste = xgb.DMatrix(teste[listaVariaveis])
    
    # Converte rotulos booleanos para inteiro (embora não seja estritamente necessário em python)
    validacao = teste[rotulo].apply(int).reset_index(drop=True)

    # Vetor de resultados
    resultados = []
    for rodada in intervaloGrade:
        resultados.append(xgBoost(xgbDados, xgbTeste, validacao, rotulo, rodada))
    dfResultado = pd.DataFrame()
    return resultados

def xgBoost(treinamento, teste, validacao, rotulo, rodadas):
    """ Executa o xgBoost e retorna tupla com número de rodadas e AUC
    """
    param = {'booster': 'dart',
             #'objective': 'binary:logistic',
             #'eta': 0.05,
             #'gamma': 0,
             'max_depth': 8,
             #'min_child_weight': 1,
             #'subsample': 1,
             #'colsample_bytree': 1
            }
    print('XGBoost, {rodadas} rodadas'.format(rodadas=rodadas))
    # Treina modelo pelo número de rodadas determinado
    modelo = xgb.train(param, treinamento, rodadas)
    
    # Computa a previsão do modelo para o conjunto de teste
    resultadoValidacao = modelo.predict(teste)

    # Cria dataframe com previsão e rótulo
    previsao = 'previsao'
    dfPrevisao = pd.DataFrame(resultadoValidacao, columns=[previsao])
    dfPrevisao[rotulo] = validacao
    
    # Ordena o dataframe pela coluna de previsao
    dfPrevisao = dfPrevisao.sort_values(by=previsao)
    # Usamos o AUC do sklearn.metrics
    # requer: from sklearn.metrics import auc
    return (rodadas, auc(dfPrevisao[previsao], dfPrevisao[rotulo]))
    

In [27]:
grade = gradeBoost(dadosTreino, variaveis, dadosValidacao, resultado, range(125, 126, 10))

XGBoost, 125 rodadas
[00:42:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=8
[00:42:51] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:42:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=8
[00:42:51] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:42:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 236 extra nodes, 0 pruned nodes, max_depth=8
[00:42:51] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:42:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=8
[00:42:52] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:42:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=8
[00:42:52] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:42:53

[00:43:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=8
[00:43:03] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=8
[00:43:03] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 302 extra nodes, 0 pruned nodes, max_depth=8
[00:43:03] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=8
[00:43:03] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 244 extra nodes, 0 pruned nodes, max_depth=8
[00:43:04] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:04] /workspace/src/tree

[00:43:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 260 extra nodes, 0 pruned nodes, max_depth=8
[00:43:19] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=8
[00:43:19] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 0 pruned nodes, max_depth=8
[00:43:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 0 pruned nodes, max_depth=8
[00:43:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 300 extra nodes, 0 pruned nodes, max_depth=8
[00:43:21] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:21] /workspace/src/tree

In [28]:
grade

[(125, 0.7552212476730347)]

In [29]:
grade = gradeBoost(dadosTreino, variaveis, dadosValidacao, resultado, range(50, 160, 1))

XGBoost, 50 rodadas
[00:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=8
[00:43:39] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=8
[00:43:39] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 236 extra nodes, 0 pruned nodes, max_depth=8
[00:43:39] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=8
[00:43:39] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=8
[00:43:39] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:40]

[00:43:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=8
[00:43:52] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=8
[00:43:52] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 302 extra nodes, 0 pruned nodes, max_depth=8
[00:43:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=8
[00:43:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 244 extra nodes, 0 pruned nodes, max_depth=8
[00:43:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:43:54] /workspace/src/tree

[00:44:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=8
[00:44:05] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 330 extra nodes, 0 pruned nodes, max_depth=8
[00:44:06] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=8
[00:44:06] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=8
[00:44:06] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=8
[00:44:06] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:07] /workspace/src/tree

[00:44:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=8
[00:44:19] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=8
[00:44:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 280 extra nodes, 0 pruned nodes, max_depth=8
[00:44:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 326 extra nodes, 0 pruned nodes, max_depth=8
[00:44:21] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=8
[00:44:21] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:21] /workspace/src/tree

[00:44:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 0 pruned nodes, max_depth=8
[00:44:33] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=8
[00:44:34] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=8
[00:44:34] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 282 extra nodes, 0 pruned nodes, max_depth=8
[00:44:35] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=8
[00:44:35] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[00:44:36] /workspace/src/tree

...

[01:40:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=8
[01:40:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:40:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=8
[01:40:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:40:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 270 extra nodes, 0 pruned nodes, max_depth=8
[01:40:54] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:40:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 356 extra nodes, 0 pruned nodes, max_depth=8
[01:40:54] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:40:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 192 extra nodes, 0 pruned nodes, max_depth=8
[01:40:55] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:40:55] /workspace/src/tree

[01:41:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 0 pruned nodes, max_depth=8
[01:41:07] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=8
[01:41:07] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 192 extra nodes, 0 pruned nodes, max_depth=8
[01:41:07] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 266 extra nodes, 0 pruned nodes, max_depth=8
[01:41:08] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=8
[01:41:08] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:08] /workspace/src/tree

[01:41:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=8
[01:41:19] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 248 extra nodes, 0 pruned nodes, max_depth=8
[01:41:19] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=8
[01:41:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 0 pruned nodes, max_depth=8
[01:41:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 0 pruned nodes, max_depth=8
[01:41:20] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:21] /workspace/src/tree

[01:41:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=8
[01:41:34] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 310 extra nodes, 0 pruned nodes, max_depth=8
[01:41:34] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 312 extra nodes, 0 pruned nodes, max_depth=8
[01:41:34] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 238 extra nodes, 0 pruned nodes, max_depth=8
[01:41:35] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 282 extra nodes, 0 pruned nodes, max_depth=8
[01:41:35] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:36] /workspace/src/tree

[01:41:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 0 pruned nodes, max_depth=8
[01:41:52] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 0 pruned nodes, max_depth=8
[01:41:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 248 extra nodes, 0 pruned nodes, max_depth=8
[01:41:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=8
[01:41:53] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 344 extra nodes, 0 pruned nodes, max_depth=8
[01:41:54] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[01:41:54] /workspace/src/tree

In [30]:
grade

[(50, 0.7344983071088791),
 (51, 0.7184089422225952),
 (52, 0.7304233610630035),
 (53, 0.7268094420433044),
 (54, 0.724725753068924),
 (55, 0.7154417932033539),
 (56, 0.7018200159072876),
 (57, 0.7109204828739166),
 (58, 0.700346976518631),
 (59, 0.6991380751132965),
 (60, 0.7277215123176575),
 (61, 0.6872818470001221),
 (62, 0.688477024435997),
 (63, 0.687017485499382),
 (64, 0.6950610280036926),
 (65, 0.6995080709457397),
 (66, 0.70403653383255),
 (67, 0.7284897267818451),
 (68, 0.7008056491613388),
 (69, 0.7158064842224121),
 (70, 0.6978052854537964),
 (71, 0.7000023871660233),
 (72, 0.47020383179187775),
 (73, 0.4702134430408478),
 (74, 0.4946853816509247),
 (75, 0.726181149482727),
 (76, 0.7306439727544785),
 (77, 0.8054626137018204),
 (78, 0.7727451920509338),
 (79, 0.7631285935640335),
 (80, 0.7658304870128632),
 (81, 0.7603559046983719),
 (82, 0.7486288994550705),
 (83, 0.757875919342041),
 (84, 0.7515316158533096),
 (85, 0.7482845783233643),
 (86, 0.7568823993206024),
 (87, 0.

In [31]:
dfGrade = pd.DataFrame({"AUC":[g[1] for g in grade], "Rodadas":[g[0] for g in grade]})

In [32]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

In [33]:
p = figure(width=900, height=600)
#p.y_range = Range1d(0,1)
p.line("Rodadas", "AUC", source=dfGrade)
show(p)

In [34]:
aucMax = dfGrade["AUC"].max()
numRodadas = dfGrade[dfGrade["AUC"] == aucMax]["Rodadas"].iloc[0]
print('AUC Máximo: {}, {} rodadas'.format(aucMax, numRodadas))

AUC Máximo: 0.8830898255109787, 148 rodadas


In [35]:
from bokeh.io import export_png
arquivoSaida = os.path.join('imagens', 'xgBoost_AUC_{rotulo}.png'.format(rotulo=resultado))
export_png(p, filename=arquivoSaida)

'/media/stelling/OS/xampp/htdocs/KDD2009/python/imagens/xgBoost_AUC_churn.png'

[](file:///media/stelling/OS/xampp/htdocs/KDD2009/python/imagens/xgBoost_AUC_churn.png')

# AUC/Rodada
![nada](imagens/xgBoost_AUC_churn.png)